In [ ]:
#! pip install plotly==5.8.2
#! pip install jupyter-dash

In [1]:
# Import required libraries
import pandas as pd
#import dash
from jupyter_dash import JupyterDash
from dash import html,dcc
from dash.dependencies import Input,Output
import plotly.express as px
import wget

In [2]:
# get dataset and read into pandas dataframe
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')
spacex_df=pd.read_csv(spacex_csv_file)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

100% [................................................................................] 2476 / 2476

In [3]:
spacex_df.tail()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
51,51,48,CCAFS SLC-40,0,4230.0,F9 FT B1032.2,FT
52,52,50,CCAFS SLC-40,0,6092.0,F9 B4 B1044,B4
53,53,52,CCAFS SLC-40,0,2647.0,F9 B4 B1039.2,B4
54,54,53,CCAFS SLC-40,1,362.0,F9 B4 B1045.1,B4
55,55,56,CCAFS SLC-40,0,5384.0,F9 B4 B1040.2,B4


In [4]:
spacex_df[spacex_df['Launch Site']=='CCAFS SLC-40'][['class']].value_counts()

class
0        4
1        3
dtype: int64

In [5]:
pd.DataFrame.(spacex_df[spacex_df['Launch Site']=='CCAFS SLC-40'][['class']].value_counts()).reset_index()

SyntaxError: invalid syntax (<ipython-input-5-ca30e8344fcd>, line 1)

In [6]:
# Create a dash application
#app = dash.Dash(__name__)
app=JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'}]+
                                             [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
                                             value='ALL',
                                             searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=None,
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown',component_property='value'))

def get_pie_chart(entered_site):
    
    if entered_site == 'ALL':
        # dataset: success counts per site
        pie_data_all=spacex_df[['Launch Site','class']].groupby('Launch Site').sum().reset_index()
        fig = px.pie(pie_data_all, values='class', names='Launch Site', title='Percentage of total successful launches')
        
    else:
        
        # dataset: success percentage for chosen site
        pie_data_single=pd.DataFrame(spacex_df[spacex_df['Launch Site']==entered_site][['class']].
                                      value_counts()).reset_index()
        pie_data_single.columns=['class','count']
        pie_data_single['Result']=['Success' if result else 'Failure' for result in pie_data_single['class']]
        fig = px.pie(pie_data_single, values='count', names='Result', title='Launches for site '+entered_site,
                    color='Result',color_discrete_map={'Success':'green','Failure':'firebrick'})
        
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
             [Input(component_id='site-dropdown',component_property='value'),
             Input(component_id='payload-slider',component_property='value')])

def get_scatter_chart(entered_site,slider_range):
    
    if entered_site=='ALL':
        # dataset: all payload mass and corresponding class
        scatter_data=spacex_df
        title='Outcome vs Payload for all launch sites'
    else:
        # dataset: payload data per launch site
        scatter_data=spacex_df[spacex_df['Launch Site']==entered_site]
        title='Outcome vs Payload for launch site '+entered_site
        
    dataset=scatter_data[['Payload Mass (kg)','class','Booster Version Category']]  
    fig=px.scatter(scatter_data,x='Payload Mass (kg)',y='class',color='Booster Version Category',
                   range_x=slider_range,range_y=[-0.5,1.5],title=title,labels={'class':'Launch Outcome'})
    fig.update_layout(yaxis = dict(tickvals = [0,1],ticktext = ['Failure','Success']))
    
    return fig

# Run the app4,3,2,1
if __name__ == '__main__':
    #app.run_server()
    app.run_server(mode='external', host='localhost', debug=False, dev_tools_ui=False, dev_tools_props_check=False)

 * Running on http://localhost:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Aug/2022 12:19:41] "GET /_alive_8343834f-6c3d-4a89-8c8d-e4fbcce54534 HTTP/1.1" 200 -


Dash app running on http://localhost:8050/


127.0.0.1 - - [27/Aug/2022 12:19:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2022 12:19:45] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2022 12:19:45] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2022 12:19:45] "GET /_favicon.ico?v=2.5.1 HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2022 12:19:45] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2022 12:19:45] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2022 12:19:45] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2022 12:19:45] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2022 12:19:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2022 12:19:47] "POST /_dash-update-component HTTP/1.1" 200 -
